In [34]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install lightkurve
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install astropy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install xgboost
!{sys.executable} -m pip install lightgbm # https://www.reddit.com/r/git/comments/sme7k4/add_specific_folder_with_submodule_to_a_repository/
!{sys.executable} -m pip install catboost

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings(action='ignore')

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached lightgbm-3.3.2.tar.gz (1.5 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [77 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib
      creating build/lib/lightg

ModuleNotFoundError: No module named 'lightgbm'

In [6]:
data = pd.read_csv('../Ansible/Data/cumulative.csv')
data

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9559,9560,10031643,K07984.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,0,0,...,-152.0,4.296,0.231,-0.189,1.088,0.313,-0.228,298.74921,46.973351,14.478
9560,9561,10090151,K07985.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,1,...,-166.0,4.529,0.035,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082
9561,9562,10128825,K07986.01,NaN,CANDIDATE,CANDIDATE,0.497,0,0,0,...,-220.0,4.444,0.056,-0.224,1.031,0.341,-0.114,286.50937,47.163219,14.757
9562,9563,10147276,K07987.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.021,0,0,1,...,-236.0,4.447,0.056,-0.224,1.041,0.341,-0.114,294.16489,47.176281,15.385


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9564 entries, 0 to 9563
Data columns (total 50 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rowid              9564 non-null   int64  
 1   kepid              9564 non-null   int64  
 2   kepoi_name         9564 non-null   object 
 3   kepler_name        2294 non-null   object 
 4   koi_disposition    9564 non-null   object 
 5   koi_pdisposition   9564 non-null   object 
 6   koi_score          8054 non-null   float64
 7   koi_fpflag_nt      9564 non-null   int64  
 8   koi_fpflag_ss      9564 non-null   int64  
 9   koi_fpflag_co      9564 non-null   int64  
 10  koi_fpflag_ec      9564 non-null   int64  
 11  koi_period         9564 non-null   float64
 12  koi_period_err1    9110 non-null   float64
 13  koi_period_err2    9110 non-null   float64
 14  koi_time0bk        9564 non-null   float64
 15  koi_time0bk_err1   9110 non-null   float64
 16  koi_time0bk_err2   9110 

## Preprocessing

In [33]:
def preprocess_inputs(df):
    df = df.copy()

    # Drop unused columns
    df = df.drop(['rowid', 'kepid', 'kepoi_name', 'kepler_name', 'koi_pdisposition', 'koi_score'], axis = 1)

    # Limit target values to CANDIDATE and CONFIRMED
    false_positive_rows = df.query("koi_disposition == 'FALSE POSITIVE'").index
    df = df.drop(false_positive_rows, axis = 0).reset_index(drop=True)

    # Drop columns with all missing values -> come back to these later (at some point)
    df = df.drop(['koi_teq_err1', 'koi_teq_err2'], axis = 1)

    # Fill remaining missing values
    df['koi_tce_delivname'] = df['koi_tce_delivname'].fillna(df['koi_tce_delivname'].mode()[0])
    for column in df.columns[df.isna().sum() > 0]:
        df[column] = df[column].fillna(df[column].mean())

    # One-hot encode koi_tce_delivname column
    delivname_dummies = pd.get_dummies(df['koi_tce_delivname'], prefix = 'delivname')
    df = pd.concat([df, delivname_dummies], axis = 1) # Add to dataset
    df = df.drop('koi_tce_delivname', axis = 1)

    # Split df into X and Y
    y = df['koi_disposition']
    X = df.drop('koi_disposition', axis = 1)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, shuffle = True, random_state = 1)

    # Scale the data | Same range of values /#/ https://www.youtube.com/watch?v=nklFC-_xi2U&t=663s
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), index = X_train.index, columns = X_train.columns)
    test = pd.DataFrame(scaler.transform(X_test), index = X_test.index, columns = X_test.columns)

    #return df
    return X_train, X_test, y_train, y_test

#X = preprocess_inputs(data)
#X

X_train, X_test, y_train, y_test = preprocess_inputs(data)
X_train

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag,delivname_q1_q16_tce,delivname_q1_q17_dr24_tce,delivname_q1_q17_dr25_tce
1673,-0.071134,-0.117116,-0.0355,-0.017742,-0.037165,-0.239706,0.239706,-0.497521,-0.000669,0.000669,...,0.162643,-0.012212,-0.045685,0.033953,-1.053449,-1.878703,-0.325469,-0.242131,-0.185756,0.312051
1239,-0.071134,-0.117116,-0.0355,-0.017742,-0.004275,-0.054711,0.054711,0.202478,0.092227,-0.092227,...,0.548779,-0.083330,-0.292382,0.077194,0.659043,0.429700,0.597714,-0.242131,-0.185756,0.312051
3589,-0.071134,-0.117116,-0.0355,-0.017742,0.174458,1.954787,-1.954787,-0.239901,0.177133,-0.177133,...,-2.636846,0.476722,3.648441,-0.313997,-1.261863,-1.319194,-1.420679,4.129990,-0.185756,-3.204607
415,-0.071134,-0.117116,-0.0355,-0.017742,-0.037962,-0.253911,0.253911,0.110849,-0.411112,0.411112,...,-0.947499,-0.096417,0.021787,0.102868,-1.024723,0.715114,1.253905,-0.242131,-0.185756,0.312051
2925,-0.071134,-0.117116,-0.0355,-0.017742,0.010035,0.559955,-0.559955,0.873931,0.966253,-0.966253,...,0.210910,0.000876,-0.066771,0.012333,-0.261806,-1.248770,0.522987,-0.242131,-0.185756,0.312051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,-0.071134,-0.117116,-0.0355,-0.017742,-0.035650,-0.206766,0.206766,-0.396396,0.736509,-0.736509,...,-1.092300,-0.080613,0.139864,0.090031,1.143456,1.093772,0.255217,-0.242131,-0.185756,0.312051
2763,-0.071134,-0.117116,-0.0355,-0.017742,-0.031798,-0.076652,0.076652,-0.276621,-0.121035,0.121035,...,-1.019900,-0.081848,0.095585,0.075842,0.628206,-1.259209,0.958113,-0.242131,-0.185756,0.312051
905,-0.071134,-0.117116,-0.0355,-0.017742,-0.026434,-0.229689,0.229689,-0.280512,-0.276861,0.276861,...,0.259177,-0.031967,-0.146894,0.025846,0.606496,-0.141000,0.904403,-0.242131,-0.185756,0.312051
3980,-0.071134,-0.117116,-0.0355,-0.017742,-0.034204,-0.235459,0.235459,-0.328916,-0.176973,0.176973,...,0.247110,-0.121605,-0.324010,0.144081,-1.268001,1.283264,-0.464025,-0.242131,-0.185756,0.312051


In [27]:
X.select_dtypes('object')

,koi_disposition
0,CONFIRMED
1,CONFIRMED
2,CONFIRMED
3,CONFIRMED
4,CONFIRMED
...,...
4536,CANDIDATE
4537,CANDIDATE
4538,CANDIDATE
4539,CANDIDATE


# Training

In [ ]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Neural Network": MLPClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
}